In [14]:
!pip install duckdb

In [15]:
import duckdb
con = duckdb.connect()

In [17]:
con.execute("""
    SELECT *
    FROM read_parquet('green_tripdata_2025-11.parquet')
    LIMIT 5
""").df()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1,74,42,1,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1,1,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1,74,42,2,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2,1,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1,83,160,1,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1,1,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1,166,127,1,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1,1,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1,166,262,1,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1,1,2.75,0.0


In [28]:
df_green =con.execute("""
    SELECT *
    FROM read_parquet('green_tripdata_2025-11.parquet')
""")
df_zone = con.execute("""
    SELECT *
    FROM read_csv('taxi_zone_lookup.csv')
""")


In [33]:
#Question 3

result = con.execute("""
    SELECT
        COUNT(*) AS num_trips
    FROM read_parquet('green_tripdata_2025-11.parquet')
    WHERE lpep_pickup_datetime >= TIMESTAMP '2025-11-01'
      AND lpep_pickup_datetime <  TIMESTAMP '2025-12-01'
      AND trip_distance <= 1
""").fetchone()

num_trips = result[0]

print(
    f"Total trips with distance less than or equal to 1 mile in november 2025 is "
    f"{num_trips}"
)

Total trips with distance less than or equal to 1 mile in november 2025 is 8007


In [35]:
#Question 4

df_result = con.execute("""
    WITH filtered_trips AS (
        SELECT
            DATE(lpep_pickup_datetime) AS pickup_day,
            trip_distance
        FROM read_parquet('green_tripdata_2025-11.parquet')
        WHERE lpep_pickup_datetime >= TIMESTAMP '2025-11-01'
          AND lpep_pickup_datetime <  TIMESTAMP '2025-12-01'
          AND trip_distance < 100
    ),
    longest_trip_by_day AS (
        SELECT
            pickup_day,
            MAX(trip_distance) AS max_trip_distance
        FROM filtered_trips
        GROUP BY pickup_day
    )
    SELECT
        pickup_day,
        max_trip_distance
    FROM longest_trip_by_day
    ORDER BY max_trip_distance DESC
    LIMIT 1
""").df()

row = df_result.iloc[0]

print(
    f"Pickup day with longest trip distance (less than 100 mile) "
    f"is: {row['pickup_day']} with distance {row['max_trip_distance']:.2f} mile."
)

Pickup day with longest trip distance (less than 100 mile) is: 2025-11-14 00:00:00 with distance 88.03 mile.


In [36]:
#Question 5

df_result = con.execute("""
    SELECT
        z.Zone AS pickup_zone,
        SUM(t.total_amount) AS total_amount
    FROM read_parquet('green_tripdata_2025-11.parquet') t
    JOIN read_csv_auto('taxi_zone_lookup.csv') z
        ON t.PULocationID = z.LocationID
    WHERE DATE(t.lpep_pickup_datetime) = DATE '2025-11-18'
    GROUP BY z.Zone
    ORDER BY total_amount DESC
    LIMIT 1
""").df()

row = df_result.iloc[0]

print(
    f"pickup zone with the largest total_amount at 18 November 2025 "
    f"is '{row['pickup_zone']}' with total {row['total_amount']:.2f}."
)

pickup zone with the largest total_amount at 18 November 2025 is 'East Harlem North' with total 9281.92.


In [37]:
#Question 6

df_result = con.execute("""
    SELECT
        z_do.Zone AS dropoff_zone,
        MAX(t.tip_amount) AS max_tip
    FROM read_parquet('green_tripdata_2025-11.parquet') t
    JOIN read_csv_auto('taxi_zone_lookup.csv') z_pu
        ON t.PULocationID = z_pu.LocationID
    JOIN read_csv_auto('taxi_zone_lookup.csv') z_do
        ON t.DOLocationID = z_do.LocationID
    WHERE t.lpep_pickup_datetime >= TIMESTAMP '2025-11-01'
      AND t.lpep_pickup_datetime <  TIMESTAMP '2025-12-01'
      AND z_pu.Zone = 'East Harlem North'
    GROUP BY z_do.Zone
    ORDER BY max_tip DESC
    LIMIT 1
""").df()

row = df_result.iloc[0]

print(
    f"drop off zone that had the largest tip from "
    f"'East Harlem North' in November 2025 is "
    f"'{row['dropoff_zone']}' with tip {row['max_tip']:.2f}."
)

drop off zone that had the largest tip from 'East Harlem North' in November 2025 is 'Yorkville West' with tip 81.89.
